In [63]:
# Imports and configs
import pandas as pd
import numpy as np
from pickle import dump, load

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, accuracy_score

import math

import re

from z3 import *
from functools import partial

tf.get_logger().setLevel('ERROR')

# Pandas-Jupyter options
pd.set_option("display.precision", 4)
pd.set_option("display.max_rows", 6)
pd.set_option("display.max_columns", 90)
pd.set_option("display.float_format", '{:,.2g}'.format)

# Disable SKLearn warnings
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

import gc
gc.disable()
gc.collect()
print("GC is enabled" if gc.isenabled() else "GC is disabled")

GC is disabled


In [2]:
# Load Data
# Filter flows
def filter_flows(flows):
    recognized_types = ["BENIGN", "Virut", "Neris", "RBot", "IRC attack", "Murlo", "Tbot", "Sogou"]
    flows = flows[flows["FlowType"].isin(recognized_types)]
    flows = flows[flows["Protocol"].isin(["TCP", "UDP", "ICMP"])]
    flows = flows.replace(np.nan, 0).replace(np.inf, 0)
    flows = flows[flows["Flow Duration"] >= 0]
    return flows

def process(flows):
    non_features = ["Flow ID", "Src IP", "Src Port", "Dst IP", "Dst Port", "Timestamp", "HostType", "FlowType", "Protocol", "FlowTerminationReason",]
    flows = flows[flows.columns.drop(list(flows.filter(regex='Idle.*|Active.*|.*/s|^Flow IAT.*|^Packet Length.*|.*Mean.*|.*Average.*')))] # Idle.*|Active.*|.*/s'
    flows_features = flows[flows.columns.drop(non_features)]
    flow_labels = flows["FlowType"].replace(to_replace=r"^(.(?<!BENIGN))*?$", value="MALICIOUS",regex=True).replace({ "BENIGN": 0, "MALICIOUS": 1 })
    flows_features = flows_features.assign(**flows_features[["Total Length of Fwd Packet", "Total Length of Bwd Packet"]].astype(np.int64))
    return flows_features, flow_labels, flows

# Load flows
flows_train = pd.read_csv('../ISCX_Botnet-Training.pcap_Flow.csv', index_col=False)
flows_test = pd.read_csv('../ISCX_Botnet-Testing.pcap_Flow.csv', index_col=False)

flows_train = filter_flows(flows_train)
flows_test = filter_flows(flows_test)

train_features, train_labels, train_flows = process(flows_train)
test_features, test_labels, test_flows = process(flows_test)

train_malicious_flows = train_flows[train_flows["FlowType"] != "BENIGN"]
train_malicious_features = train_features.loc[train_malicious_flows.index]
train_malicious_labels = train_labels.loc[train_malicious_flows.index]

train_benign_flows = train_flows[train_flows["FlowType"] == "BENIGN"]
train_benign_features = train_features.loc[train_benign_flows.index]
train_benign_labels = train_labels.loc[train_benign_flows.index]

test_malicious_flows = test_flows[test_flows["FlowType"] != "BENIGN"]
test_malicious_features = test_features.loc[test_malicious_flows.index]
test_malicious_labels = test_labels.loc[test_malicious_flows.index]

In [3]:
# Import Scaler (set feature count)
feature_count = 50
scaler = None

def scale_data(flows_features, scaler_file=f'archive_{feature_count}/scaler.pkl', load_scaler=False):
    global scaler
    if not scaler:
        if load_scaler:
            scaler = load(open(scaler_file, 'rb'))
        else:
            scaler = MinMaxScaler().fit(flows_features)
            dump(scaler, open(scaler_file, 'wb'))
    return scaler.transform(flows_features)

In [4]:
# Split sample by classification result (set threshold)
threshold = 0.5
margin = 0.05

# Get correctly classified
def get_correct_samples_with_margin(model, features, labels, threshold, margin):
    # Scale features
    scaled_features = scale_data(features)

    # Predict
    predictions = model.predict(scaled_features, verbose=0)
    prediction_labels = [math.ceil(prediction[0]) if prediction[0] > threshold else math.floor(prediction[0]) for prediction in predictions]
    selected = [True if prediction[0] > threshold and prediction[0] < threshold + margin else False for prediction in predictions]
    
    labels = labels.copy().to_frame()
    labels["PredictedFlowType"] = prediction_labels
    correct_labels = labels[selected][labels["FlowType"] == labels["PredictedFlowType"]]
    correct_features = features[selected].loc[correct_labels.index]
    return correct_features, correct_labels.drop(columns=["PredictedFlowType"])

model = tf.keras.models.load_model("dnn_base.h5")

train_correct_features, train_correct_labels = get_correct_samples_with_margin(model, train_features, train_labels, threshold, margin)
test_correct_features, test_correct_labels = get_correct_samples_with_margin(model, test_features, test_labels, threshold, margin)

test_correct_malicious_labels = test_correct_labels[test_correct_labels.index.isin(test_malicious_labels.index)]
test_correct_malicious_features = test_correct_features.loc[test_correct_malicious_labels.index]

train_correct_malicious_labels = train_correct_labels[train_correct_labels.index.isin(train_malicious_labels.index)]
train_correct_malicious_features = train_correct_features.loc[train_correct_malicious_labels.index]

print("Train mal.: %s samples" % len(train_malicious_features))
print("Train correct mal. (%s < conf. < %s): %s samples (%.2f percent of total)" % (threshold, threshold + margin, len(train_correct_malicious_features), len(train_correct_malicious_features) / len(train_malicious_features)*100))

print("Test mal.: %s samples" % len(test_malicious_features))
print("Test correct mal. (%s < conf. < %s): %s samples (%.2f percent of total)" % (threshold, threshold + margin, len(test_correct_malicious_features), len(test_correct_malicious_features) / len(test_malicious_features)*100))

# SANITY CHECK
# Check if for all features prediction is greater than threshold and less than margin
def confidence_margin(model, features, threshold, margin):
    scaled_features = scale_data(features)
    predictions = model.predict(scaled_features, verbose=0)
    selected = [True if prediction[0] > threshold and prediction[0] < threshold + margin else False for prediction in predictions]
    return all(selected)

assert confidence_margin(model, train_correct_malicious_features, threshold, margin)
assert confidence_margin(model, test_correct_malicious_features, threshold, margin)

Train mal.: 109305 samples
Train correct mal. (0.5 < conf. < 0.55): 836 samples (0.76 percent of total)
Test mal.: 87960 samples
Test correct mal. (0.5 < conf. < 0.55): 351 samples (0.40 percent of total)


In [81]:
# Verifier

# Constants
ReLU = 0
Linear = 1
Const0 = 2
Unknown = -1

inv_sigmoid = np.vectorize(lambda x: np.log(x) - np.log(1 - x))
z3_real = np.vectorize(Real)
z3_real_val = np.vectorize(RealVal)

def _z3_relu(x):
    return If(x > 0, x, 0)

z3_relu = np.vectorize(_z3_relu)
z3_simplify = np.vectorize(_z3_relu)

# -- Based on previous layer bounds calculates the bounds for the next layer (without activation)
def _next_layer_bounds(weights, biases, input_bounds):
    next_min_bounds = []
    next_max_bounds = []
    prev_count, next_count = weights.shape
    min_bounds, max_bounds = input_bounds
    # for each neuron in the next layer compute the bounds
    for i in range(next_count):
        # compute the bounds for the i-th neuron
        next_min_bound = 0
        next_max_bound = 0
        for j in range(prev_count):
            if weights[j][i] > 0:
                next_min_bound += weights[j][i] * min_bounds[j]
                next_max_bound += weights[j][i] * max_bounds[j]
            else:
                next_min_bound += weights[j][i] * max_bounds[j]
                next_max_bound += weights[j][i] * min_bounds[j]
        next_min_bound += biases[i]
        next_max_bound += biases[i]
        next_min_bounds.append(next_min_bound)
        next_max_bounds.append(next_max_bound)

    return next_min_bounds, next_max_bounds

def _optimize_layer_bounds(layer_activation, input_bounds):
    min_bounds, max_bounds = input_bounds
    for j in range(len(min_bounds)):
        if layer_activation[j] == ReLU:
            if min_bounds[j] >= 0:
                layer_activation[j] = Linear
            elif max_bounds[j] <= 0:
                layer_activation[j] = Const0
            min_bounds[j] = max(min_bounds[j], 0)
            max_bounds[j] = max(max_bounds[j], 0)
        elif layer_activation[j] == Linear:
            min_bounds[j] == 0
        elif layer_activation[j] == Const0:
            min_bounds[j] = 0
            max_bounds[j] = 0

def _create_activation_matrix(model, skip_output_activation=True):
    activation = []
    for i, layer in enumerate(model.layers):
        if layer.activation == tf.keras.activations.relu:
            activation.append([ReLU] * layer.output_shape[1])
        elif layer.activation == tf.keras.activations.linear:
            activation.append([Linear] * layer.output_shape[1])
        else:
            if skip_output_activation and layer == model.layers[-1]:
                activation.append([Linear] * layer.output_shape[1])
            else:
                print("ERROR: Activation function is not supported!")
                activation.append([Linear] * layer.output_shape[1])
    return activation

def _optimize_layer_activations(layer_activation, X, x, constraints):
    # 1. Compute neurons value for the current layer based on the previous one without activation
    # 2. If the value > 0, check if on the input range it is always > 0:
    #    if yes, then consider the neuron activation as linear
    #    if no, then neuron activation is ReLU
    # 3. If the value < 0, check if on the input range it is always < 0
    #    if yes, then consider the neuron activation as constant 0
    #    if no, then neuron activation is ReLU
    
    # Build optimized z3 representation of the current layer
    # MAYBE: reuse the boundaries known for the previous layer
    # TODO: parallelize the SMT tasks for each neuron
    for i in range(len(X)):
        if layer_activation[i] == ReLU:
            s = Solver()
            for constraint in constraints:
                s.add(constraint)
            if x[i] > 0:
                # Check if on the input range it is always > 0:
                s.add(Not(X[i] >= 0))
                if s.check() == unsat:
                    layer_activation[i] = Linear
                    # MAYBE: this "> 0" fact can later be used to optimize the next layer
                else:
                    X[i] = z3_relu(X[i])
                    x[i] = max(x[i], 0)
            elif x[i] < 0:
                # Check if on the input range it is always < 0
                s.add(Not(X[i] <= 0))
                if s.check() == unsat:
                    X[i] = RealVal(0.) # Consider the neuron activation as constant 0
                    x[i] = 0
                    layer_activation[i] = Const0
                else:
                    X[i] = z3_relu(X[i])
                    x[i] = max(x[i], 0)
            else:
                print("Unexpected!")
                X[i] = z3_relu(X[i])
                x[i] = max(x[i], 0)
        if layer_activation[i] == Const0:
            X[i] = RealVal(0.)
            x[i] = 0
        elif layer_activation[i] == Unknown:
            print("Activation function is not supported")

    return input, constraints

def _absolute_eps_ball_constraints(x, eps, input, feat_indexes, freeze_zero_features=True):
    # TODO: handle case when eps > 1
    # TODO: handle case when x[i] == 0
    # NOTE: here we assumes features are in [0, +inf] range. Remove "max" and/or add clamping if needed otherwise
    if feat_indexes is None:
        constraints = [And(RealVal(max(x[i] - eps, 0)) < input[i], input[i] < RealVal(x[i] + eps)) for i in range(len(x))]
    else:
        if freeze_zero_features:
            constraints = [
                And(input[i] >= RealVal(max(x[i] - eps, 0)), input[i] <= RealVal(x[i] + eps)) 
                if x[i] != 0 
                else input[i] == RealVal(0.) 
                for i in range(len(x))
            ]
        else:         
            constraints = [
                And(input[i] >= RealVal(max(x[i] - eps, 0)), input[i] <= RealVal(x[i] + eps)) 
                for i in feat_indexes
            ]

    return constraints    

def _relative_eps_ball_constraints(x, eps, input, feat_indexes):
    # TODO: handle case when eps > 1
    if feat_indexes is None:
        constraints = [And(x[i] * RealVal(1 - eps) <= input[i], input[i] <= x[i] * RealVal(1 + eps)) for i in range(len(x))]
    else:
        # NOTE: No epsilon-ball for 0-valued features
        constraints = [
            And(input[i] >= RealVal(x[i] * (1 - eps)), input[i] <= RealVal(x[i] * (1 + eps)))
            for i in feat_indexes
        ]
    return constraints

# Compute input bounds (asumes inputs are non-negative)
def _absolute_eps_input_bounds(x, eps, feat_indexes):
    if feat_indexes is None:
        min_bounds, max_bounds = np.maximum(0, x - eps), x + eps
    else:
        min_bounds, max_bounds = x.copy(), x.copy()
        min_bounds[feat_indexes] = np.maximum(0, min_bounds[feat_indexes] - eps)
        max_bounds[feat_indexes] += eps
    return min_bounds, max_bounds

def _relative_eps_input_bounds(x, eps, feat_indexes):
    # TODO: handle case when eps > 1
    if feat_indexes is None:
        min_bounds, max_bounds = x * (1 - eps), x * (1 + eps)
    else:
        min_bounds, max_bounds = x.copy(), x.copy()
        min_bounds[feat_indexes] *= (1 - eps)
        max_bounds[feat_indexes] *= (1 + eps)
    return min_bounds, max_bounds

def _sample_feasible_inputs(input, constraints, count):
    samples = []
    solver = SolverFor("QF_LRA")
    solver.add(constraints)

    for _ in range(count):
        if solver.check() == sat:
            m = solver.model()
            sample = np.array([
                float(m[i].as_fraction()) 
                if isinstance(m[i], RatNumRef) 
                else float(i.as_fraction())
                for i in input
            ])
            samples.append(sample)
            # Add a new constraint that blocks the current solution
            solver.add(Or([d() != m[d] for d in m.decls()]))
        else:
            assert False, "Constraints are infeasible"
            break
    return samples  

def z3_var(name):
    # Replace all non-alphanumeric characters with underscores
    name = re.sub(r'\W', '_', name).lower()
    return Real(name)

# -- Produces Z3 representation of the DNN model optimized around the given input
def to_z3_graph_optimized_around(model, input, x, eps, constraints, input_bounds, skip_output_activation=True, eps_ball_relative=True):
    # Construct activation matrix for each layer
    activation = _create_activation_matrix(model, skip_output_activation)

    X = input
    for layer_index, layer in enumerate(model.layers):
        params = layer.get_weights()
        
        # Layer numerical representation (no activation yet)
        w = params[0]
        b = params[1]
        x = x @ w + b

        # Layer Z3 representation (no activation yet)
        W = z3_real_val(w)
        B = z3_real_val(b)
        X = X @ W + B

        input_bounds = _next_layer_bounds(w, b, input_bounds)
        _optimize_layer_bounds(activation[layer_index], input_bounds)

        if skip_output_activation and layer == model.layers[-1]:
            continue
        
        _optimize_layer_activations(activation[layer_index], X, x, constraints)
    
    return X, activation

samples = []
x_d = None
input = None
def eps_ball_verify(model, X, expected_label, eps, threshold, features, free_features, extra_constraints=[], eps_ball_relative=True, debug=True):
    # TODO: add timer
    # TODO: try parallellizing the SMT tasks for neurons in the same layer

    global samples
    global x_d
    global input

    feat_indexes = [features.get_loc(f) for f in free_features] if free_features else None
    batch_size = 25
    result = [None] * len(X)
    relu_count = []
    linear_count = []
    const_count = []
    for i, x in enumerate(X):
        x_d = x
        if feat_indexes is None:
            input = z3_real([z3_var(f) for f in features])
        else:
            input = [z3_var(f) if f in free_features else RealVal(x[f_i]) for f_i, f in enumerate(features)]

        if eps_ball_relative:
            constraints = _relative_eps_ball_constraints(x, eps, input, feat_indexes)
            input_bounds = _relative_eps_input_bounds(x, eps, feat_indexes)
        else:
            constraints = _absolute_eps_ball_constraints(x, eps, input, feat_indexes)
            input_bounds = _absolute_eps_input_bounds(x, eps, feat_indexes)

        constraints += extra_constraints
        output, activation = to_z3_graph_optimized_around(model, input, x, eps, constraints, input_bounds, eps_ball_relative)

        # Count ReLU activations after optimization (just for statistics)
        relu_count.append(sum([sum([1 for a in layer if a == ReLU]) for layer in activation]))

        s = SolverFor("QF_LRA")
        # s = Solver()
        s.add(constraints)
        s.add(Not(output[0] > inv_sigmoid(threshold) if expected_label == 1 else output[0] <= inv_sigmoid(threshold)))
        result[i] = s.check() == unsat

        if i != 0 and i % batch_size == 0:
            print(f"Processed {i} samples (eps = {eps}): {sum(result[:i])} verified, {i - sum(result[:i])} failed")

        if debug:
            # === SANITY ASSERTIONS === 
            tolerance = 2e-5
            if not result[i]:
                m = s.model()
                if feat_indexes is None:
                    counterexample = np.array([float(m[var].as_fraction()) for var in input])
                else:
                    counterexample = x.copy()
                    for feature in feat_indexes:
                        counterexample[feature] = float(m[input[feature]].as_fraction())
                
                tf_out = model.predict(np.expand_dims(counterexample, axis=0), verbose=0)[0][0]
                z3_out = tf.sigmoid(float(m.evaluate(output[0]).as_fraction())).numpy()

                if expected_label == 1:
                    assert tf_out <= threshold + tolerance, f"TF output exceeds threshold {tf_out} > {threshold}"
                else:
                    assert tf_out + tolerance > threshold, f"TF output doesn't exceed threshold {tf_out} <= {threshold}"

                assert abs(tf_out - z3_out) < tolerance, f"TF and Z3 ouputs does not match: {tf_out} != {z3_out}"
            else:
                # take several random points from the input constraints space and check that they are classified correctly
                # perturb x by a small amount and check that it is classified correctly
                # TODO: IMPORTANT! fix inter-feature scaling
                sample_count = 0
                samples = _sample_feasible_inputs(input, constraints, count=sample_count)
                
                for sample in samples:
                    tf_out = model.predict(np.expand_dims(sample, axis=0), verbose=0)[0][0]
                    if expected_label == 1:
                        assert tf_out + tolerance > threshold, "Verification passed, but a perturbed sample was miscalssified"
                    else:
                        assert tf_out < threshold + tolerance, "Verification passed, but a perturbed sample was miscalssified"

    avg_relu_count = sum(relu_count) / len(relu_count)
    print(f"Processed {len(X)} samples (eps = {eps}): {sum(result)} verified, {len(X) - sum(result)} failed; Avg. ReLU count: {avg_relu_count}")
    return result

In [69]:
def denormalize(feature, feature_name):
    # Get the index of the feature
    features = scaler.get_feature_names_out()
    
    # Find the index of feature_name in feature_index
    index = np.where(features == feature_name)
    min_val = scaler.data_min_[index].flatten()[0]
    max_val = scaler.data_max_[index].flatten()[0]

    return feature * (max_val - min_val) + min_val

def normalize(feature, feature_name):
    # Get the index of the feature
    features = scaler.get_feature_names_out()
    
    # Find the index of feature_name in feature_index
    index = np.where(features == feature_name)
    min_val = scaler.data_min_[index].flatten()[0]
    max_val = scaler.data_max_[index].flatten()[0]
    return (feature - min_val) / (max_val - min_val)

# Test normalization
# for i in range(len(train_features)):
#     scaled_value_1 = scaler.transform([train_features.iloc[i]])[0][0]
#     scaled_value_2 = normalize(train_features.iloc[i]["Flow Duration"], "Flow Duration")
#     print(i)
#     assert abs(scaled_value_1 - scaled_value_2) < 1e-9, f"Normalization failed: {scaled_value_1} != {scaled_value_2}"

In [70]:
# Define free features and constraints
MALICIOUS = 1
BENIGN = 0

free_features = [
    "Flow Duration",
    # "Fwd Packet Length Max", "Fwd Packet Length Min", "Fwd Packet Length Std",
    # "Bwd Packet Length Max", "Bwd Packet Length Min", "Bwd Packet Length Std",
    "Fwd IAT Total", "Fwd IAT Std", "Fwd IAT Max", "Fwd IAT Min", 
    "Bwd IAT Total", "Bwd IAT Std", "Bwd IAT Max", "Bwd IAT Min"
]

# constraints = [
#     z3_var("Fwd IAT Max") >= z3_var("Fwd IAT Min"),
#     z3_var("Bwd IAT Max") >= z3_var("Bwd IAT Min")
# ]

constraints = [
    denormalize(z3_var("Fwd IAT Max"), "Fwd IAT Max") >= denormalize(z3_var("Fwd IAT Min"), "Fwd IAT Min"),
    denormalize(z3_var("Bwd IAT Max"), "Fwd IAT Max") >= denormalize(z3_var("Bwd IAT Min"), "Bwd IAT Min")
]

In [71]:
# Verify a set of points for base model
model = tf.keras.models.load_model("dnn_base.h5")
epsilon = 0.1

result = eps_ball_verify(
    model=model,
    X=scale_data(train_correct_malicious_features),
    expected_label=MALICIOUS,
    eps=epsilon,
    threshold=threshold,
    features=train_correct_malicious_features.columns,
    free_features=free_features,
    extra_constraints=constraints, 
    debug=True)

_ = gc.collect()
# 15m 7.5s

Processed 25 samples (eps = 0.1): 21 verified, 4 failed
Processed 50 samples (eps = 0.1): 38 verified, 12 failed
Processed 75 samples (eps = 0.1): 54 verified, 21 failed
Processed 100 samples (eps = 0.1): 73 verified, 27 failed
Processed 125 samples (eps = 0.1): 94 verified, 31 failed
Processed 150 samples (eps = 0.1): 118 verified, 32 failed
Processed 175 samples (eps = 0.1): 138 verified, 37 failed
Processed 200 samples (eps = 0.1): 160 verified, 40 failed
Processed 225 samples (eps = 0.1): 177 verified, 48 failed
Processed 250 samples (eps = 0.1): 198 verified, 52 failed
Processed 275 samples (eps = 0.1): 219 verified, 56 failed
Processed 300 samples (eps = 0.1): 239 verified, 61 failed
Processed 325 samples (eps = 0.1): 260 verified, 65 failed
Processed 350 samples (eps = 0.1): 283 verified, 67 failed
Processed 375 samples (eps = 0.1): 306 verified, 69 failed
Processed 400 samples (eps = 0.1): 330 verified, 70 failed
Processed 425 samples (eps = 0.1): 353 verified, 72 failed
Proces

In [72]:
# Verify a set of points for base model
model = tf.keras.models.load_model("dnn_base.h5")
epsilon = 0.05

result = eps_ball_verify(
    model=model,
    X=scale_data(train_correct_malicious_features),
    expected_label=MALICIOUS,
    eps=epsilon,
    threshold=threshold,
    features=train_correct_malicious_features.columns,
    free_features=free_features,
    extra_constraints=constraints, 
    debug=True)

_ = gc.collect()
# 6m 31.9s

Processed 25 samples (eps = 0.05): 21 verified, 4 failed
Processed 50 samples (eps = 0.05): 38 verified, 12 failed
Processed 75 samples (eps = 0.05): 54 verified, 21 failed
Processed 100 samples (eps = 0.05): 73 verified, 27 failed
Processed 125 samples (eps = 0.05): 96 verified, 29 failed
Processed 150 samples (eps = 0.05): 120 verified, 30 failed
Processed 175 samples (eps = 0.05): 140 verified, 35 failed
Processed 200 samples (eps = 0.05): 163 verified, 37 failed
Processed 225 samples (eps = 0.05): 181 verified, 44 failed
Processed 250 samples (eps = 0.05): 202 verified, 48 failed
Processed 275 samples (eps = 0.05): 223 verified, 52 failed
Processed 300 samples (eps = 0.05): 244 verified, 56 failed
Processed 325 samples (eps = 0.05): 267 verified, 58 failed
Processed 350 samples (eps = 0.05): 292 verified, 58 failed
Processed 375 samples (eps = 0.05): 316 verified, 59 failed
Processed 400 samples (eps = 0.05): 340 verified, 60 failed
Processed 425 samples (eps = 0.05): 365 verified,

In [83]:
# Verify a set of points for base model
model = tf.keras.models.load_model("dnn_base.h5")
epsilon = 0.15

result = eps_ball_verify(
    model=model,
    X=scale_data(train_correct_malicious_features),
    expected_label=MALICIOUS,
    eps=epsilon,
    threshold=threshold,
    features=train_correct_malicious_features.columns,
    free_features=free_features,
    extra_constraints=constraints, 
    debug=True)

_ = gc.collect()
# 92m 22.7s

Processed 25 samples (eps = 0.15): 21 verified, 4 failed
Processed 50 samples (eps = 0.15): 37 verified, 13 failed
Processed 75 samples (eps = 0.15): 53 verified, 22 failed
Processed 100 samples (eps = 0.15): 72 verified, 28 failed
Processed 125 samples (eps = 0.15): 93 verified, 32 failed
Processed 150 samples (eps = 0.15): 117 verified, 33 failed
Processed 175 samples (eps = 0.15): 136 verified, 39 failed
Processed 200 samples (eps = 0.15): 157 verified, 43 failed
Processed 225 samples (eps = 0.15): 174 verified, 51 failed
Processed 250 samples (eps = 0.15): 194 verified, 56 failed
Processed 275 samples (eps = 0.15): 215 verified, 60 failed
Processed 300 samples (eps = 0.15): 235 verified, 65 failed
Processed 325 samples (eps = 0.15): 256 verified, 69 failed
Processed 350 samples (eps = 0.15): 279 verified, 71 failed
Processed 375 samples (eps = 0.15): 301 verified, 74 failed
Processed 400 samples (eps = 0.15): 325 verified, 75 failed
Processed 425 samples (eps = 0.15): 347 verified,

93m 25s - 711 verified, 125 failed